In [0]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [2]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
imagePaths = list(paths.list_images('/drive/My Drive/AAA_Data Science/Semestre 2/T2 - Deep Learning/deep_learning/Group_Project/original_images'))
data = []
labels = []

In [4]:
# loop over the image paths
for imagePath in tqdm(imagePaths):
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

100%|██████████| 5805/5805 [1:09:08<00:00,  1.40it/s]


In [0]:
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [0]:
# # Saving data and labels as a pkl file
pickle.dump(data, open("/drive/My Drive/Colab Notebooks/data.pkl", "wb"))
pickle.dump(labels, open("/drive/My Drive/Colab Notebooks/labels.pkl", "wb"))

In [0]:
# data = pickle.load(open("/drive/My Drive/Colab Notebooks/data.pkl", "rb"))
# labels = pickle.load(open("/drive/My Drive/Colab Notebooks/labels.pkl", "rb"))

In [0]:
# perform one-hot encoding on the labels
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [0]:
# partition the data into training(60%),validation(20%) and testing(20%)
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)
(trainX, validationX, trainY, validationY) = train_test_split(trainX, trainY,test_size=0.25, stratify=trainY, random_state=42)

In [0]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [26]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [0]:
# loop over the layers in the base model and freeze them. Now, I'm unfreezing the last 8 layers.
for layer in baseModel.layers[:-8]: 
	layer.trainable = False

In [0]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = MaxPooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel) # apagar
headModel = Dropout(0.5)(headModel)
headModel = Dense(25, activation="softmax")(headModel)

In [0]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [0]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [0]:
# compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [32]:
# train the head of the network
history = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(validationX, validationY),
	validation_steps=len(validationX) // BS,
	epochs=EPOCHS)

Epoch 1/20
108/108 [==============================] - 45s 413ms/step - loss: 4.5198 - accuracy: 0.1736 - val_loss: 2.2775 - val_accuracy: 0.3196
Epoch 2/20
108/108 [==============================] - 43s 398ms/step - loss: 2.4355 - accuracy: 0.3118 - val_loss: 1.9717 - val_accuracy: 0.4083
Epoch 3/20
108/108 [==============================] - 43s 399ms/step - loss: 2.0368 - accuracy: 0.3981 - val_loss: 1.6946 - val_accuracy: 0.4918
Epoch 4/20
108/108 [==============================] - 43s 399ms/step - loss: 1.7775 - accuracy: 0.4720 - val_loss: 1.7464 - val_accuracy: 0.4694
Epoch 5/20
108/108 [==============================] - 43s 398ms/step - loss: 1.5616 - accuracy: 0.5355 - val_loss: 1.5537 - val_accuracy: 0.5444
Epoch 6/20
108/108 [==============================] - 43s 401ms/step - loss: 1.4465 - accuracy: 0.5677 - val_loss: 1.5344 - val_accuracy: 0.5409
Epoch 7/20
108/108 [==============================] - 43s 397ms/step - loss: 1.3224 - accuracy: 0.5914 - val_loss: 1.4484 - val_ac

In [0]:
# make predictions on the testing set
predictions = model.predict(testX, batch_size=BS)

In [0]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predictions = np.argmax(predictions, axis=1)

In [0]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predictions,target_names=lb.classes_))

In [0]:
# serialize the model to disk
model.save('/drive/My Drive/Colab Notebooks/Models/CNN_Pretrained_V5', save_format="h5")

In [0]:
# plot the training accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig('/drive/My Drive/Colab Notebooks/Plots/CNN_Pretrained_V5_ACC.png')

In [0]:
# plot the training loss
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('/drive/My Drive/Colab Notebooks/Plots/CNN_Pretrained_V5_Loss.png')